In [145]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

import time

In [146]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [147]:
data_path = "./data"
train_data_path = "./DataSet/train"
test_data_path = "./DataSet/test"

In [148]:
# Hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001
num_classes = 6

In [149]:
dataset = datasets.ImageFolder(root=train_data_path, transform=transforms.Compose([ transforms.Resize((256, 256)),  # Görüntüleri 256x256 boyutuna yeniden boyutlandır
                                                                                    transforms.CenterCrop(224),
                                                                                    transforms.ToTensor()])         )
loader = DataLoader(train_data_path, batch_size=1, shuffle=False)

In [150]:
# Calculate mean and standard deviation for normalization
'''mean = 0.0
std = 0.0
for img, _ in loader:
    mean = mean + img.mean(dim=[0, 2, 3])
    std = std + img.std(dim=[0, 2, 3])
    
mean = mean / len(loader.dataset)
std = std / len(loader.dataset)
print(f"Mean: {mean}, Standard Deviation: {std}")'''

'mean = 0.0\nstd = 0.0\nfor img, _ in loader:\n    mean = mean + img.mean(dim=[0, 2, 3])\n    std = std + img.std(dim=[0, 2, 3])\n    \nmean = mean / len(loader.dataset)\nstd = std / len(loader.dataset)\nprint(f"Mean: {mean}, Standard Deviation: {std}")'

In [151]:
# DATA PREPROCESSING
transform = transforms.Compose([ transforms.Resize((256, 256)), 
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=(0.5, 0.5, 0.5) , std=(0.5, 0.5, 0.5)) ]) 

In [152]:
# Dataset ve DataLoader
'''dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
loader = DataLoader(train_data_path, batch_size=1, shuffle=True)  # 1 görüntü ile kontrol

# Normalize edilmiş görüntüyü kontrol et
for img, _ in loader:
    print("Mean:", img.mean(dim=[0, 2, 3]))  # Kanal başına ortalama
    print("Standard Deviation:", img.std(dim=[0, 2, 3]))  # Kanal başına std
    break  # İlk görüntü ile kontrol yeterlidir'''

'dataset = datasets.ImageFolder(root=train_data_path, transform=transform)\nloader = DataLoader(train_data_path, batch_size=1, shuffle=True)  # 1 görüntü ile kontrol\n\n# Normalize edilmiş görüntüyü kontrol et\nfor img, _ in loader:\n    print("Mean:", img.mean(dim=[0, 2, 3]))  # Kanal başına ortalama\n    print("Standard Deviation:", img.std(dim=[0, 2, 3]))  # Kanal başına std\n    break  # İlk görüntü ile kontrol yeterlidir'

In [153]:
# SPLIT DATA
# Eğitim ve test verilerini yükleme
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [154]:
# DEFINE the CNN Architecture
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [155]:
# Initialize the model
model = CNNet().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [156]:
# Training loop
start = time.time()

total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device) # 64 x 3 x 32 x 32
        labels = labels.to(device) # 64 x 10

        # Forward pass
        outputs = model(images) # 64 x 10
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')

end = time.time()
print(f"Training finished in {end - start} sec!")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x93312 and 512x512)

AĞAÇ TEMELLİ YÖNTEM

In [174]:
# LOAD DATA

# Görseller ve etiketler için boş listeler
images = []
labels = []

# Sınıf isimlerini al (örneğin, 'Arborio', 'Basmati' gibi)
classes = os.listdir(data_path)
class_indices = {cls_name: i for i, cls_name in enumerate(classes)}  # Etiketleme için

# Görselleri ve etiketleri yükle
for cls_name in classes:
    cls_path = os.path.join(data_path, cls_name)
    for img_name in os.listdir(cls_path):
        img_path = os.path.join(cls_path, img_name)

        # Görselleri yükle ve boyutunu kontrol et
        image = Image.open(img_path).convert("L")  # Gri tonlamalı
        image = image.resize((256, 256))  # Eğer boyut farklıysa yeniden boyutlandır

        # Görselleri ve etiketleri listeye ekle
        images.append(np.array(image))  # Görselleri numpy dizisine dönüştür
        labels.append(class_indices[cls_name])  # Etiketi ekle


In [175]:
# Görseller ve etiketleri NumPy dizilerine dönüştür
images = np.array(images)  # Görseller (n, 250, 250)
labels = np.array(labels)  # Etiketler (n, )

print(f"Toplam Görsel: {len(images)}")
print(f"Görsellerin Boyutu: {images.shape}")
print(f"Etiketlerin Boyutu: {labels.shape}")


Toplam Görsel: 186
Görsellerin Boyutu: (186, 256, 256)
Etiketlerin Boyutu: (186,)


In [176]:
# SCALE DATA
images = images / 255.0

In [177]:
images[0].max()

np.float64(1.0)

In [178]:
# Öznitelik çıkarma: Histogram
def extract_features(image):
    return np.histogram(image.ravel(), bins=256, range=(0, 256))[0]

# Tüm görüntüler için öznitelik çıkarımı
features = np.array([extract_features(image) for image in images]) 

In [191]:
# SPLIT DATA

# Veriyi eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42) # öznitelik ve etiketlere göre

print(f"Eğitim Verisi: {X_train.shape}, Test Verisi: {X_test.shape}")

Eğitim Verisi: (148, 256), Test Verisi: (38, 256)


In [192]:
# Random Forest modelini oluşturma ve eğitme
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

start = time.time()
rf_model.fit(X_train, y_train)
end = time.time()

print(f"Training in Random Forest finished in {end - start} sec!")

Training in Random Forest finished in 0.044523000717163086 sec!


In [193]:
# Test verisi üzerinde tahmin
y_pred = rf_model.predict(X_test)

# Doğruluk hesaplama
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Karışıklık matrisi
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Detaylı sınıflandırma raporu
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.50
Confusion Matrix:
[[4 1 2 0 1 0]
 [0 2 0 0 1 0]
 [0 2 0 3 0 0]
 [0 0 0 8 0 0]
 [0 1 0 0 5 0]
 [1 0 1 6 0 0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.33      0.67      0.44         3
           2       0.00      0.00      0.00         5
           3       0.47      1.00      0.64         8
           4       0.71      0.83      0.77         6
           5       0.00      0.00      0.00         8

    accuracy                           0.50        38
   macro avg       0.39      0.50      0.41        38
weighted avg       0.41      0.50      0.42        38



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  